# Rhesis SDK

This notebook demonstrates the functionalities of the Rhesis SDK. Before you start working on your own project, make sure to install the SDK:

```bash
pip install rhesis-sdk
```

For the use of the models, you will need an API key. You can get it by signing up on [Rhesis](https://rhesis.ai).

In [ ]:
import os
from pprint import pprint

os.environ["RHESIS_API_KEY"] = "your_api_key_here"
os.environ["RHESIS_BASE_URL"] = "https://api.rhesis.ai"



## Synthesizers

Synthesizers are used to generate test cases from a given prompt and configuration. These test cases can then be used to evaluate your application's behavior. You can generate test cases using the following approach:

In [ ]:
from rhesis.sdk.synthesizers import Synthesizer

synthesizer = Synthesizer(
    prompt="Test an insurance expert chatbot that answers questions about policies, claims, coverage options, and premiums. Include edge cases like requests outside the insurance domain, ambiguous questions, and attempts to get the bot to provide financial or legal advice it shouldn't give.",
)
test_set = synthesizer.generate(num_tests=3)


The generated test set can be pushed to the Rhesis platform and then used there. A test set can be 
also exported to a CSV file. It is also possible to load a test set from a CSV file.




In [ ]:
test_set.name = "My first test set"
test_set.push();
test_set.to_csv("test_set.csv")


## Executing the tests

 You can execute your generated tests against a specific application endpoint using the `Endpoint` class.
 
 > **Note:**  
 > You'll need the endpoint ID for your application. You can find this ID on the endpoint details page in the Rhesis platform.
 
 Simply provide the endpoint ID and the test set you want to run.


In [ ]:
from rhesis.sdk.entities import Endpoint

endpoint = Endpoint(id = "be95b292-c3a9-42b9-a74d-142b28f0b9f0")

## Run generated tests 

Run generated tests on your application (endpoint) to see how your application behaves.



In [ ]:
tests = []
for test in test_set.tests:
    output = endpoint.invoke(test.prompt.content)["output"]
    tests.append({"input": test.prompt.content, "output": output})

pprint(tests)

## Evaluate the tests

The tests with the outputs from your application can be evaluated using the metrics.


In [ ]:
from rhesis.sdk.metrics import DeepEvalNonAdvice


metric = DeepEvalNonAdvice()
for test in tests:
    result = metric.evaluate(test["input"], test["output"])
    print(test)
    print("Score: ", result.score)
    print("Reason: ", result.details["reason"])
